In [1]:
import numpy as np
from mlfz.nn import Model, precast
from mlfz.nn.tensor import Tensor, mean_squared_error, sum
from mlfz.nn.tensor.functional import *

In [2]:
from functools import partial

def _finite_diff(f, x, h=1e-8):
    x = np.asarray(x, dtype=float)
    grad = np.zeros_like(x)

    for idx in np.ndindex(x.shape):
        x_forward = x.copy()
        x_backward = x.copy()
        x_forward[idx] += h
        x_backward[idx] -= h
        grad[idx] = (f(x_forward) - f(x_backward)) / (2 * h)

    return grad.reshape(x.shape)

In [3]:
x = Tensor(np.array([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]]))
ys = [
    Tensor(2),
    Tensor(np.array([[7.0, 8.0], [9.0, 10.0], [11.0, 12.0]])),
    Tensor(np.array([[1], [2], [3]])),
    Tensor(np.array([1, 2])),
]

f = lambda x, y: (x * y).sum()

In [4]:
y = Tensor(np.array([[1], [2], [3]]))
z = f(x, y)
z.backward()

y.backwards_grad

array([[ 3.,  7., 11.],
       [ 3.,  7., 11.],
       [ 3.,  7., 11.]])

In [5]:
_finite_diff(partial(f, x.value), ys[2].value)

array([[ 2.99999989],
       [ 6.99999987],
       [10.99999984]])

In [6]:
x = Tensor(np.array([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]]))
ys = [
    Tensor(2),
    Tensor(np.array([[7.0, 8.0], [9.0, 10.0], [11.0, 12.0]])),
    Tensor(np.array([[1], [2], [3]])),
    Tensor(np.array([1, 2])),
]

fs = [
    # lambda x, y: (x + y).sum(),
    # lambda x, y: (y + x).sum(),
    lambda x, y: (x * y).sum(),
    # lambda x, y: (y * x).sum(),
    # lambda x, y: (x**y).sum(),
    # lambda x, y: (x / y).sum(),
]

for f in fs:
    for y in ys:
        z = f(x, y)
        z.backward()
        print(np.allclose(
            x.backwards_grad, _finite_diff(partial(f, y=y.value), x.value), 1e-2
        ))
        print(np.allclose(
            y.backwards_grad, _finite_diff(partial(f, x.value), y.value), 1e-2
        ))

True
True
True
True
True
False
True
True
